In [ ]:
import calitp.magics
import pandas as pd
from calitp import *
from calitp.tables import tbl
from siuba import *
import utils

pd.options.display.max_columns = 100
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

## Black Cat

In [ ]:
# Read in Black Cat: 5311 and 5310 recipients with projects in past 2 years
df_bc = pd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/5311-5310/5311_5310_applicants.parquet", engine="auto"
)

In [ ]:
# Clean up organization names
df_bc = utils.organization_cleaning(df_bc, 'organization_name')

In [ ]:
# Clean up names to match Transit Stacks 
df_bc["organization_name"] = df_bc["organization_name"].replace(
    {
    
        "Tuolumne County Transit Agency": "Tuolumne County Transit Agency",
        "Omnitrans": "OmniTrans",
        'Eastern Sierra Transit Authority':'Eastern Sierra Transit Authority Community Routes',
        'Fresno County Rural Transit Agency':'Fresno County Rural Transit',
        'El Dorado County Transit Authority': 'El Dorado Transit',
        'Palo Verde Valley Transit Agency':'Palos Verdes Peninsula Transit Authority',
        'Kern Regional Transit':'Kern Transit',
        'Riverside Transit Agency':'Riverside Transit',
        'Tuolumne County Transit Agency ':'Tuolumne County Transit',
    }
)

## Transit Stacks, Service Components

In [ ]:
# Get service components data 
%%sql -o df_service_components
SELECT 
  service_name,
  product_name,
  component_name,
FROM cal-itp-data-infra.mart_transit_database.dim_service_components

In [ ]:
components_wanted = [
    "Mobile ticketing",
    "Cash Farebox",
    "Fare card system",
    "Ticket Vending Machines",
    "Contactless Payment Validators",
    "Payment processor",
]

In [ ]:
# Filter for only fare related components
df_service_components2 = df_service_components[
    df_service_components["component_name"].isin(components_wanted)
]

In [ ]:
df_service_components2 = utils.organization_cleaning(df_service_components2, "service_name")

In [ ]:
len(df_service_components2)

In [ ]:
# Merge transit stacks with BC
m1 = pd.merge(
    df_bc,
    df_service_components2,
    how="left",
    left_on=["organization_name"],
    right_on=["service_name"],
    indicator=True,
    validate='1:m'
)

In [ ]:
m1.shape

In [ ]:
m1.head()

In [ ]:
# df_bc['organization_name'].sort_values().unique().tolist()